## Run the pipeline:

In [1]:
from core.meta import MetaInterface

In [ ]:
d = MetaInterface(
    project_config_filepath="test_data/project_config.yaml",
    project_name="test_data",
)

In [ ]:
d.add_recording_config("test_data/Server_structure/Calibrations/221222/recording_config_221222.yaml")

In [ ]:
d.initialize_meta_config()
d.remove_recordings()

In [ ]:
d.create_recordings(recreate_undistorted_plots=True)

In [ ]:
d.synchronize_recordings(verbose=True)

In [ ]:
d.create_calibrations(
    ground_truth_config_filepath="test_data/ground_truth_config.yaml", recreate_undistorted_plots=True
)

In [ ]:
d.synchronize_calibrations(verbose=True)

In [ ]:
d.exclude_markers(all_markers_to_exclude_config_path = "test_data/markers_to_exclude_config.yaml", verbose=False)

In [ ]:
d.calibrate(verbose=2, calibrate_optimal=True)

In [ ]:
d.triangulate_recordings()

In [ ]:
d.normalize_recordings(normalization_config_path="test_data/normalization_config.yaml", verbose=True)

In [ ]:
d.add_triangulated_csv_to_database("test.csv")

In [ ]:
d.objects["triangulation_recordings_objects"][
    "206_F2-63_220922_OTE"
].create_triangulated_video("test_video", "test_data/video_plotting_config_OTE.yaml")

#### Add metadata to database

In [ ]:
# create empty database
import pandas as pd

df = pd.DataFrame(
    {},
    columns=[
        "recording",
        "date",
        "session_id",
        "paradigm",
        "subject_id",
        "group_id",
        "batch",
        "trial_id",
    ],
)
df.to_csv("test.csv", index=False)

In [ ]:
# add group id, session id, batch, trial id
import pandas as pd
def add_database_metadata(recording, overwrite=False) -> None:
    old_df = pd.read_csv(database, dtype="str")
    if (
        old_df.loc[
            old_df["recording"] == recording,
            ["session_id", "group_id", "batch", "trial_id"],
        ]
        .isna()
        .any(axis=1)
        .values[0]
        or overwrite
    ):
        subject_id = old_df.loc[old_df["recording"] == recording, "subject_id"].values[
            0
        ]
        recording_date = old_df.loc[old_df["recording"] == recording, "date"].values[0]

        try:
            batch = groups.loc[groups["subject_id"] == subject_id, "batch"].values[0]
        except IndexError:
            raise ValueError(f"Found no group identity for {subject_id}!")
            
        group = groups.loc[groups["subject_id"] == subject_id, "group_id"].values[0]
        
        try:
            session = (
                time.loc[time["batch"] == int(batch),]
                .loc[time["date"] == int(recording_date), "session_id"]
                .values[0]
            )
        except IndexError:
            raise ValueError(f"Found no session identity for {subject_id} {recording_date}!")
            
        try:
            trial_id = (
                time.loc[time["batch"] == int(batch),]
                .loc[time["date"] == int(recording_date), "trial_id"]
                .values[0]
            )
        except IndexError:
            raise ValueError(f"Found no trial identity for {subject_id} {recording_date}!")

        old_df.loc[
            old_df["recording"] == recording,
            ["group_id", "batch", "session_id", "trial_id"],
        ] = (group, batch, session, trial_id)
        old_df.to_csv(database, index=False)
    else:
        print("metadata was already in csv!")
    return old_df


time = pd.read_excel("time.xlsx")
groups = pd.read_excel("groups.xlsx")
database = "test.csv"

for recording in pd.read_csv(database, dtype="str")["recording"].values:
    add_database_metadata(recording, overwrite=False)

### The meta .yaml file contains all information stored in d.meta

In [ ]:
d.export_meta_to_yaml(filepath="test_data/test_data.yaml")

#### To inspect the files, we recommend https://www.treedoc.org/, which creates a tree-like structured view for .yaml files

In [ ]:
#d.objects['triangulation_recordings_objects']['206_F2-63_220922_OTE'].metadata_from_videos['Bottom'].fps